In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('/home/server/gli-data-science/')
import ds_db

import numpy as np
import gensim
from multiprocessing import  Pool

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)



In [2]:
q = '''

select tbtdr_ponta_user, tbtdr_tbto_id, tbtdr_rating, tbtdr_review 
from tb_transaction_delivery_rating ttdr 
where tbtdr_review notnull and tbtdr_review != ''
and tbtdr_created_date between '2021-06-01' and '2021-07-25'
'''

con, cur = ds_db.connect_prd_order_3()
df_re = pd.read_sql(q, con)


con.close()

You are connected


In [3]:
df_re = df_re[df_re['tbtdr_rating'].isin([1,2])]

In [5]:
from nlp_id.lemmatizer import Lemmatizer 
from nlp_id.stopword import StopWord 
from nlp_id.tokenizer import Tokenizer 

import re
import nltk
import string

def basic_clean_single(text):
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    lemmatizer = Lemmatizer() 
    
    stopword = StopWord() 
    stopwords = stopword.get_stopword() 
    
    tokenizer = Tokenizer() 
    words = tokenizer.tokenize(text)
    #return [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    #return [lemmatizer.lemmatize(word) for word in words]
    return words

def basic_clean(df):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    df['tbtdr_review_token'] = df['tbtdr_review'].apply(basic_clean_single)
    return df


# processed_docs = list(df_re['tbtdr_review'].apply(basic_clean))

In [6]:
def parallelize_dataframe(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

processed_docs = parallelize_dataframe(df_re, basic_clean)

In [7]:
processed_docs.head()

,tbtdr_ponta_user,tbtdr_tbto_id,tbtdr_rating,tbtdr_review,tbtdr_review_token
233,9990013350229355,6103590,2.0,Pembayaran Sulit,"[Pembayaran, Sulit]"
387,9990010138638665,5550827,1.0,"pesanan ini dibatalkan karna waktu kirim tdk sesuai dg estimasi, tetapi poin saya tetap terpotong","[pesanan, ini, dibatalkan, karna, waktu, kirim, tdk, sesuai, dg, estimasi, tetapi, poin, saya, tetap, terpotong]"
391,9990013149303785,1225562,2.0,"hanya whatsapp konfirmasi pesanan tanpa pemberitahuan produk ready semua atau ga dan hanya confirm shareloc, sesampainya pesanan malah yang dibutuhkan tidak ada. salut lah kecewanya","[hanya, whatsapp, konfirmasi, pesanan, tanpa, pemberitahuan, produk, ready, semua, atau, ga, dan, hanya, confirm, shareloc, sesampainya, pesanan, malah, yang, dibutuhkan, tidak, ada, salut, lah, kecewanya]"
495,9990013226669030,7101935,2.0,Produk rusak,"[Produk, rusak]"
518,9990013357708115,7101676,1.0,"udh belanja, kok poin nya gk ada?","[udh, belanja, kok, poin, nya, gk, ada]"


In [8]:

processed_docs = list(processed_docs['tbtdr_review_token'])
dictionary = gensim.corpora.Dictionary(processed_docs)

In [25]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 7, 
                                   id2word = dictionary,                                    
                                   passes = 20,
                                   workers = 3)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
lda_model.print_topics()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.029*"di" + 0.027*"tidak" + 0.019*"bisa" + 0.016*"dikirim" + 0.014*"digunakan" + 0.013*"Voucher" + 0.013*"yg" + 0.011*"produk" + 0.009*"order" + 0.008*"ada"'),
 (1,
  '0.040*"di" + 0.023*"yg" + 0.021*"kurir" + 0.015*"saya" + 0.013*"tidak" + 0.013*"antar" + 0.013*"nya" + 0.013*"barang" + 0.012*"ada" + 0.011*"dan"'),
 (2,
  '0.033*"Produk" + 0.030*"rusak" + 0.022*"tidak" + 0.015*"nya" + 0.013*"saya" + 0.008*"Kurir" + 0.007*"yang" + 0.007*"lagi" + 0.007*"di" + 0.007*"ada"'),
 (3,
  '0.015*"barang" + 0.015*"saya" + 0.014*"foto" + 0.013*"jam" + 0.009*"tdk" + 0.009*"di" + 0.008*"pesan" + 0.007*"sudah" + 0.007*"yg" + 0.007*"penerima"'),
 (4,
  '0.281*"Pengiriman" + 0.279*"Lama" + 0.014*"tidak" + 0.013*"jam" + 0.012*"Promo" + 0.012*"berlaku" + 0.006*"pagi" + 0.005*"pesan" + 0.005*"ada" + 0.004*"lama"'),
 (5,
  '0.015*"jam" + 0.014*"kurang" + 0.012*"di" + 0.012*"sudah" + 0.011*"lama" + 0.010*"dan" + 0.010*"saya" + 0.009*"ini" + 0.008*"mau" + 0.007*"tapi"'),
 (6,
  '0.083*"tidak" + 0.06

In [27]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, bow_corpus, dictionary)
lda_viz

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.100984 -0.180323       1        1  37.881843
1     -0.075265 -0.002207       2        1  14.130477
4      0.330518 -0.031042       3        1  12.880340
0     -0.039615  0.011386       4        1  10.540244
2     -0.038460  0.005230       5        1   9.591397
3     -0.035793  0.107120       6        1   8.295102
5     -0.040400  0.089835       7        1   6.680598, topic_info=                Term        Freq       Total Category  logprob  loglift
47        Pengiriman  420.000000  420.000000  Default  30.0000  30.0000
46              Lama  418.000000  418.000000  Default  29.0000  29.0000
39            Produk  232.000000  232.000000  Default  28.0000  28.0000
8            pesanan  284.000000  284.000000  Default  27.0000  27.0000
11            sesuai  222.000000  222.000000  Default  26.0000  26.0000
97            dengan  213.000000  213.000000  Default  25.0000  25.0000
40             rusak   42.000000   42.000000  Default  24.0000  24.0000
56                di  203.000000  203.000000  Default  23.0000  23.0000
36             tidak  466.000000  466.000000  Default  22.0000  22.0000
76               jam   63.000000   63.000000  Default  21.0000  21.0000
162           kurang   63.000000   63.000000  Default  20.0000  20.0000
71              bisa   62.000000   62.000000  Default  19.0000  19.0000
10              saya  149.000000  149.000000  Default  18.0000  18.0000
187          dikirim   93.000000   93.000000  Default  17.0000  17.0000
29            produk   68.000000   68.000000  Default  16.0000  16.0000
172            kurir   61.000000   61.000000  Default  15.0000  15.0000
193            sudah   79.000000   79.000000  Default  14.0000  14.0000
125        digunakan   24.000000   24.000000  Default  13.0000  13.0000
123          Voucher   21.000000   21.000000  Default  12.0000  12.0000
44               nya   80.000000   80.000000  Default  11.0000  11.0000
122            Promo   31.000000   31.000000  Default  10.0000  10.0000
168            antar   35.000000   35.000000  Default   9.0000   9.0000
124          berlaku   31.000000   31.000000  Default   8.0000   8.0000
63                yg  118.000000  118.000000  Default   7.0000   7.0000
205             foto   21.000000   21.000000  Default   6.0000   6.0000
213              mau   39.000000   39.000000  Default   5.0000   5.0000
48            barang  136.000000  136.000000  Default   4.0000   4.0000
12               tdk   32.000000   32.000000  Default   3.0000   3.0000
310             lama   22.000000   22.000000  Default   2.0000   2.0000
237           sampai   68.000000   68.000000  Default   1.0000   1.0000
11            sesuai  215.333238  222.005961   Topic1  -3.0116   0.9402
97            dengan  205.138676  213.324340   Topic1  -3.0601   0.9316
8            pesanan  270.707230  284.002428   Topic1  -2.7827   0.9228
51          diterima   26.954506   28.836963   Topic1  -5.0896   0.9032
428         ternyata    9.205924    9.895284   Topic1  -6.1639   0.8985
477            dapat    4.675278    5.350840   Topic1  -6.8415   0.8357
490               gr    4.674817    5.350731   Topic1  -6.8416   0.8357
686            gopay    4.673791    5.350508   Topic1  -6.8418   0.8355
215        mengantar    4.672403    5.350396   Topic1  -6.8421   0.8352
1247             isi    4.671749    5.350344   Topic1  -6.8422   0.8351
612       diperbaiki    3.766007    4.441605   Topic1  -7.0577   0.8057
186             buah    3.765155    4.441430   Topic1  -7.0580   0.8055
1241           telah    3.764789    4.441380   Topic1  -7.0581   0.8054
96            dahulu    3.764580    4.441300   Topic1  -7.0581   0.8054
796         terlebih    3.764269    4.441211   Topic1  -7.0582   0.8053
871             dtng    3.763220    4.440982   Topic1  -7.0585   0.8051
1108           namun    3.758683    4.440779   Topic1  -7.0597   0.8039
1110         te